In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/2019admith.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

23


In [4]:
docs

[Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2019-10-16T13:27:34+09:00', 'moddate': '2019-10-16T13:30:02+09:00', 'trapped': '/False', 'source': './example_data/2019admith.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}, page_content='〒701－0206\u3000岡山市南区箕島１５００番地\nＴＥＬ\u3000０８６－２８２－６３３６  \u3000 \u3000\nＵＲＬ\u3000https://www.okayama-h.ed.jp\n令 和 ２  年 度\n生 徒 募 集 要 項\n学校法人\u3000関 西 学 園\nみし ま\n岡山 高等学 校\nこの募集要項は，出願から入学説明会までの事項について記載していますので，出願後\nも大切に保管しておいてください。'),
 Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2019-10-16T13:27:34+09:00', 'moddate': '2019-10-16T13:30:02+09:00', 'trapped': '/False', 'source': './example_data/2019admith.pdf', 'total_pages': 23, 'page': 1, 'page_label': '2'}, page_content='ページ\n目\u3000\u3000\u3000\u3000\u3000次\n生徒募集要項\nⅠ\u3000入試概要\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\

In [5]:
print(f"{docs[0].page_content}\n")
print(docs[0].metadata)

〒701－0206　岡山市南区箕島１５００番地
ＴＥＬ　０８６－２８２－６３３６  　 　
ＵＲＬ　https://www.okayama-h.ed.jp
令 和 ２  年 度
生 徒 募 集 要 項
学校法人　関 西 学 園
みし ま
岡山 高等学 校
この募集要項は，出願から入学説明会までの事項について記載していますので，出願後
も大切に保管しておいてください。

{'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2019-10-16T13:27:34+09:00', 'moddate': '2019-10-16T13:30:02+09:00', 'trapped': '/False', 'source': './example_data/2019admith.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

45

In [7]:
from dotenv import load_dotenv
load_dotenv()  # .env ファイルの中身を os.environ に読み込む

True

In [8]:
import getpass
import os

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [9]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2) #埋め込みベクトルの次元数を比べる
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.03834369033575058, -0.008342096582055092, -0.0037314766086637974, 0.004695278592407703, 0.0012617340544238687, 0.009214728139340878, -0.036676570773124695, 0.0302685908973217, -0.012099621817469597, -0.0056362878531217575]


In [10]:
print(all_splits[0].page_content)
print(all_splits[1].page_content)

〒701－0206　岡山市南区箕島１５００番地
ＴＥＬ　０８６－２８２－６３３６  　 　
ＵＲＬ　https://www.okayama-h.ed.jp
令 和 ２  年 度
生 徒 募 集 要 項
学校法人　関 西 学 園
みし ま
岡山 高等学 校
この募集要項は，出願から入学説明会までの事項について記載していますので，出願後
も大切に保管しておいてください。
ページ
目　　　　　次
生徒募集要項
Ⅰ　入試概要　　　　　　　　　　　　　　　　　　　　　　　　２
Ⅱ　出願手続きについて　　　　　　　　　　　　　　　　　　　３
Ⅲ　受験上の注意　　　　　　　　　　　　　　　　　　　　　　４
Ⅳ　合否判定について　　　　　　　　　　　　　　　　　　　　５
Ⅴ　合否結果通知について　　　　　　　　　　　　　　　　　　５
Ⅵ　試験会場の地図　　　　　　　　　　　　　　　　　　　　　５
　　Ⅶ　願書記入上の注意　　　　　　　　　　　　　　　　　　　　６
［出願書類］　入学願書・入学検定料納入書・入学確約書
　　　　　　受験票
　　　　　　自己紹介書　　　　　　　　　　　　　　　　　　　　
入学手続き及び関連事項
Ⅰ　入学手続きについて　　　　　　　　　　　　　　　　　　　７
Ⅱ　入寮手続きについて　　　　　　　　　　　　　　　　　　　７
Ⅲ　岡高スカラー手続きについて　　　　　　　　　　　　　　　７
Ⅳ　入学説明会について　　　　　　　　　　　　　　　　　　　８
Ⅴ　特別講習について　　　　　　　　　　　　　　　　　　　　８
Ⅵ　特別講習期間中の宿泊について（男子）　　　　　　　　　　 ８
Ⅶ　初年度納入金・寮納入費について　　　　　　　　　　　　　９
Ⅷ　入学後の優遇制度について　　　　　　　　　　　　　　　　９
Ｑ＆Ａ　　　　　　　　　　　　　　　　　　　　　　　　　　　　　10
［入学手続き書類］　特待生申請書　　　　　　　　　　　　　　　　
　　　　　　　　　振込依頼書
　　 【資料】「高等学校等就学支援金」 ・ 「岡山県私立高等学校納付金減免補助金」 について
岡山高等学校の広報行事についてのご案内
－１－－１－
種　　類
Ｓスカラー
Ａスカラー
Ｂスカラー
入　学　金
150,000円
150,000円
学　　　費
年額708,000円
　　708,000円


In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings) # 埋め込みからベクトルストアを生成

In [13]:
ids = vector_store.add_documents(documents=all_splits)

In [14]:
print(ids)

['1d5a9e30-7a60-42e4-81d0-9c62c86ad027', '94ede53f-c84e-4752-a337-e0d87bff9577', '6cc22433-fab4-4b66-aa7d-73f24ec008ae', '352d69f3-8837-49ff-a8cf-c0d3d69e89a5', '398cdf8a-2a42-4603-bef6-b47e327cfb5f', '26378d22-2944-401c-ae36-24087000a14f', 'd7674219-b1ca-4962-bbba-07c31b0ea346', '9b20272c-fc58-42f8-8013-7f28ef74d01c', '164cd9d4-51db-4f49-bf39-ac537c41303e', '42d78b74-d293-459d-b868-2a5299c64403', '2a395934-302b-440c-ac73-a3a69c690673', '3f5f2e4d-fd85-4d67-842d-8eb0b84212b8', '371bf13e-6e13-4b2f-9e75-0c499710f1ba', '83fedd1a-7f74-4ae4-943d-71a9f15e6ffe', '6e0d9cf2-12f0-4476-8d48-1ce21f33e8bd', 'd02f3754-9bf0-4e10-b23a-5e154745edf0', '3937b736-3eb5-432e-af4b-72cd81d432c5', 'ba22ffc6-2337-4172-b4a1-2fa6cdad4443', 'fa44e892-945b-46d1-aa12-301e94dba860', 'eb521dcd-6bbf-470b-9ea9-d4082f16ed0c', 'b7aaa99b-9a5f-48e4-8562-489799ea0b3a', '7f44971f-33b9-4e9c-af5b-f253ca829a46', 'e139cfd5-8026-446e-8424-7831e280825a', '9d20a7b9-e44a-4d15-9832-e7a4f7be7ece', '11fb7c07-5bd1-4b2e-a0cf-8d530555e07c',

In [15]:
results = vector_store.similarity_search(
    "何歳の人がこの高校に入れますか？"
)

print(results[0])

page_content='令和２年度岡山高等学校入学願書【選抜１期】〈１月30日実施〉
TEL　　　  　　 －　　　   　－
〒      　－
都道
府県
郡
市
志願者
氏　名
入寮希望の有無（男子のみ）
保護者
氏　名
塾　名
ふりがな
ふりがな
東大 ・ 国立医学部
コース
難関大
コース第１志望
第２志望
出願種類入試区分
学　　歴
現住所
生年月日 平成　　　 年　　　 月　　　 日生
　　　　　年３月　　　　　　　 立　　　　　　　　　　中学校 　卒業見込 ・ 卒　業
塾名［　　　　　　　　　　　　　　　　］　　教室名［　　　　　　　　　　　　　］
塾名［　　　　　　　　　　　　　　　　］　　教室名［　　　　　　　　　　　　　］
性　　別 男  ●      女
志願者の写真貼付
上半身・無　帽
最近３ヶ月以内
タテ４㎝
ヨコ３㎝
専　願 ・ 併　願
専　願 ・ 併　願　  ・
・
難関大コース　　　　専　願　　・　　併　願
有　　●　　無
難関大
コース な　し
志願者
との続柄
※欄には記入しない
※　一般入試
※　調査書入試
入学検定料納入書
※
入学検定料
受験番号
志願者
氏　名
男 
●   
女
１３,０００円
領収印
調査書
入試
一般入試
切り離さないでください
平成
令和
検定取得級 □証明書のコピーを
　同封してください。
特記事項
漢検　　　　級　　　　　　　　年　　　月
数検　　　　級　　　　　　　　年　　　月
英検　　　　級　　　　　　　　年　　　月
平成
令和
平成
令和
平成
令和
TEL　　　  　　 －　　　   　－
都道
府県
郡
市
〒      　－在学・出身
中学校住所
（合否通知先）' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2019-10-16T13:27:34+09:00', 'moddate': '2019-10-16T13:30:02+09:00', 'trapped': '/False', 'source': './example_data/2019admith.pdf', 'total_pages': 23

In [31]:
results = vector_store.similarity_search(
    "体育学科のポリシーは？"
)

print(results[0])

page_content='第１章 学修 
－ 7 － 
体育学
部では、学部のディプロマ・ポリシーに従って、豊かな人間性、幅広い教養、基礎的学習能力、健康・
スポーツ科学に関する専門的知識、運動技能、健康・スポーツの指導ができる実践力、キャリア形成に向けて
の総合的能力を身に付けることを目的に、大きく下記の３つの科目区分に分けて、学科ごとに体系的なカリキ
ュラムを編成する。
○教養
科目、○専門基礎科目、○コア科目
教養教
育では、幅広い内容の科目を履修する一方で、専門教育（専門基礎科目、コア科目）においては、ス
ポーツや健康に関する専門知識と技能及びキャリアに関する多様な科目を履修する。 
各科目
共に、資質や能力を総合的に判定し、それぞれが成績にどのように反映されるか、評価の配分割合を
シラバスに明記している。学習成果として、 「卒業研究」や「課題レポート」 、 「教職実践演習」において全ての
ディプロマ・ポリシーを満たすことが出来ているかの最終的な確認・判定を行う。
  体育
学科 
 体育学
科では、ディプロマ・ポリシーに従って、以下の方針に基づいてカリキュラムを編成し、実施する。 
１）
豊かな人間性、幅広い教養と実践力を身に付けるために、教養科目として、 「言語・文化領域」 、 「汎用的技
能領域」 、 「人間・表現領域」 、 「数理領域」 、 「社会領域」 、 「キャリア領域」の６つのカテゴリーに区分して科
目を編成し、 「キャリア領域」でのフレッシュマンセミナーや基礎ゼミナールでは、尐人数でのメンター制度
を採用し、学生が主体的な学びを実践できるように配慮する。 
２）専門
基礎科目は、体育学の基礎科目を配置し、健康増進、体力の向上、競技力向上、教育力の向上に貢献
できる基礎的知識や現代社会において果たす教育・スポーツの役割に関して多面的な学習を行う。
３）コア
科目は、学生のキャリアに応じた「フィジカルコーチ」 、 「スポーツ科学」 、 「スポーツマネジメント」 、
「教員養成」 、 「公務員」の５つのカテゴリーに区分した科目を編成し、各領域における専門科目の体系的な
学習を行うと共に、これらを実践できる力を養う。
実技
に関する科目は、スポーツ・武道及び体育・健康づくりに関する基礎的な技術力を養うとともに、発' metadata={'produ

In [32]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='第２章 免許・資格取得に必要な科目履修方法 
－ 55 － 
 (2) 受験資格認定要件科目一覧  
 
＜体育学科＞ 
No. 科目区分 授業科目の名称 配当年次 単位数 
1 コア 運動器の解剖と機能Ⅰ 1 2 
2 コア トレーニング演習Ⅰ(基礎) 1 2 
3 専門 運動生理学 2 2 
4 専門 スポーツバイオメカニクス 2 2 
5 専門 スポーツ栄養学 2 2 
6 専門 トレーニング論Ⅰ（基礎） 2 2 
7 コア フィットネスプログラム論 2 2 
8 専門 救急処置※ 2 2 
9 コア トレーニング論Ⅱ（応用） 3 2 
10 コア トレーニング指導実習 3 2 
   ※1 「救急処置」は、 「救急処置（CSCS） 」を履修すること。 
 
＜健康科学科＞ 
No. 科目区分 授業科目の名称 配当年次 単位数 
1 他学科 運動器の解剖と機能Ⅰ 1 2 
2 専門 運動生理学 2 2 
3 専門 スポーツバイオメカニクス 2 2 
4 専門 スポーツ栄養学 2 2 
5 専門 トレーニング論Ⅰ（基礎） 2 2 
6 他学科 フィットネスプログラム論 2 2 
7 専門 トレーニング演習 2 2 
8 コア 救急処置※ 2 2 
9 専門 トレーニング論Ⅱ（応用） 3 2 
10 他学科 トレーニング指導実習 3 1 
   ※1 「救急処置」は、 「救急処置（CSCS） 」を履修すること。 
 
 
 (3) 受験までの流れ    
 
 
 
 
 
 
 
 
 
 
 
 
 
 
受
験
申
し
込
み 
願
書
受
理
通
知 
試
験
会
場
の
予
約 
受
験
（結
果
発
表
） 
約２週間 
120日間以内に受験 
資
格
認
定
証
の
発
行 
大学で一括申請 
卒業後に 
協会より送付' metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-04-15T07:17:31+00:00', 'source': './example_data/risyu_removed.pdf', 'total_pages': 73, 'page': 52, 'page_label':

In [33]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("卒業に必要な単位は何単位？")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6808915431189415

page_content='できます。履修の条件等は「他学部・他学科履修等に関する内規」を確認してください。
本学他学部及び他学科において履修した単位は、24単位までを修得単位として卒業要件単位に算入すること
ができます。
(4) 単位のルール
大学の授業は、 「単位」の考え方をもとにして行われています。 「単位」とは、授業科目の修得に必要な学習量
をはかる基準となるものです。開講する授業科目にはそれぞれ単位数が定められており、この単位を修得するた
めには、所定の時間を学習して試験その他の方法により合格しなければなりません。卒業の判定はこの修得した
単位数に基づいて行われます。 
また、卒業のために必要な単位数は124単位以上となっていますが、数字上で124単位を満たしていても、各学
科で定める科目区分ごとに必要単位を満たさなければなりません。なお、卒業に必要な授業科目区分ごとの修得
すべき単位数については後述します。 
●単位の計算方法
本学では、次のように定めています。
学 則 第24条 
  授業科目の単位数は、１単位の授業科目を45時間の学修を必要とする内容をもって構成することを標準
とし、授業の方法に応じ、当該授業による教育効果、授業時間外に必要な学修等を考慮して、次の基準に
より単位数を計算するものとする。 
(1) 講義、演習、及び講義・演習については、15時間の授業をもって１単位とする。
(2) 実験、実習、及び実技については、30時間の授業をもって１単位とする。
２ 前項の規定にかかわらず、卒業研究等の授業科目については、学修の成果を評価して単位を授与するこ
とが適切と認められる場合には、これらに必要な学修等を考慮して、単位数を定めることができる。' metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-04-15T07:17:31+00:00', 'source': './example_data/risyu_removed.pdf', 'total_pages': 73, 'page': 2, 'page_label': '3', 'start_index': 83

In [35]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("体育学科のポリシーは？")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6492197579223559

page_content='第１章 学修 
－ 7 － 
体育学
部では、学部のディプロマ・ポリシーに従って、豊かな人間性、幅広い教養、基礎的学習能力、健康・
スポーツ科学に関する専門的知識、運動技能、健康・スポーツの指導ができる実践力、キャリア形成に向けて
の総合的能力を身に付けることを目的に、大きく下記の３つの科目区分に分けて、学科ごとに体系的なカリキ
ュラムを編成する。
○教養
科目、○専門基礎科目、○コア科目
教養教
育では、幅広い内容の科目を履修する一方で、専門教育（専門基礎科目、コア科目）においては、ス
ポーツや健康に関する専門知識と技能及びキャリアに関する多様な科目を履修する。 
各科目
共に、資質や能力を総合的に判定し、それぞれが成績にどのように反映されるか、評価の配分割合を
シラバスに明記している。学習成果として、 「卒業研究」や「課題レポート」 、 「教職実践演習」において全ての
ディプロマ・ポリシーを満たすことが出来ているかの最終的な確認・判定を行う。
  体育
学科 
 体育学
科では、ディプロマ・ポリシーに従って、以下の方針に基づいてカリキュラムを編成し、実施する。 
１）
豊かな人間性、幅広い教養と実践力を身に付けるために、教養科目として、 「言語・文化領域」 、 「汎用的技
能領域」 、 「人間・表現領域」 、 「数理領域」 、 「社会領域」 、 「キャリア領域」の６つのカテゴリーに区分して科
目を編成し、 「キャリア領域」でのフレッシュマンセミナーや基礎ゼミナールでは、尐人数でのメンター制度
を採用し、学生が主体的な学びを実践できるように配慮する。 
２）専門
基礎科目は、体育学の基礎科目を配置し、健康増進、体力の向上、競技力向上、教育力の向上に貢献
できる基礎的知識や現代社会において果たす教育・スポーツの役割に関して多面的な学習を行う。
３）コア
科目は、学生のキャリアに応じた「フィジカルコーチ」 、 「スポーツ科学」 、 「スポーツマネジメント」 、
「教員養成」 、 「公務員」の５つのカテゴリーに区分した科目を編成し、各領域における専門科目の体系的な
学習を行うと共に、これらを実践できる力を養う。
実技
に関する科目は、スポーツ・武道及び体育・健康づくりに関する基礎的な技術力

In [36]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "労働衛生管理者の資格について教えて",
        "保育実習について教えて",
    ],
)

[[Document(id='275a21b7-476f-4e6d-958e-d90e81034acb', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-04-15T07:17:31+00:00', 'source': './example_data/risyu_removed.pdf', 'total_pages': 73, 'page': 69, 'page_label': '70', 'start_index': 0}, page_content='第２章 免許・資格取得に必要な科目履修方法 \n－ 72 － \n13．第一種衛生管理者免許 \n(1) 資格の内容\n対象学科：体育・健康科学・現代経営（平成30年度入学生から適用） \n第一種衛生管理者は、企業における「労働災害の防止」 「危害防止基準の確立」 「責任体制の明確化」 「自主活動\nの促進」 「労働者の安全と健康の確保」 「快適な職場環境形成」などを目的に、従業員の健康管理・健康的な職場\n環境づくりの専門家です。 \n従業員50名以上の事業所では、この資格者の常駐が義務づけられており、また同免許保持者は全ての業種にお\nいてその業務に当たることができます。 \n一般的には厚生労働大臣の指定する指定試験機関が実施する国家試験に合格する必要がありますが、本学では\n指定科目を修得することで試験免除となります。指定科目単位を修得して卒業後に申請すれば、同免許を取得で\nきます。 \n(2) 修得すべき科目と単位数\n本学において第一種衛生管理者免許の取得を希望する場合は、下記の表に示されている開講科目のすべてを卒\n業までに履修し単位を修得しなければなりません。 \n※体育学科・健康科学科・現代経営学科の所属学生であることが条件。\n※国家試験免除のため、下記３科目（労働衛生学、労働法規、労働安全衛生法）の受講までに、以下の要件に満\nたしていない者は当該３科目（労働衛生学、労働法規、労働安全衛生法）の受講が許可されないので、注意す\nること。 （要件を満たしていない者が履修した場合は、自動的に履修削除となります。 ）\n３科目（労働衛生学、労働法規、労働安全衛生法）